# KS - hiPCA

In this notebook we replicate the Kolgomorov-Smirnov hiPCA index as described in (Zhu et al, 2023). We also capture all the experiments made in pursue to obtain the best result for the CAMDA 2024 challenge

First we import the necessary libraries to run this notebook

In [74]:
import pandas as pd
import numpy as np
from scipy.stats import kstest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import chi2, norm
from sklearn.metrics import balanced_accuracy_score

Now we need to read the data

In [75]:
taxonomy = pd.read_csv('../../DataSets/CAMDA/taxonomy.txt', sep = '\t', index_col = 0)
metadata = pd.read_csv('../../DataSets/CAMDA/metadata.csv')

We select the samples which have most of the species identified

In [76]:
good_samples = []
for c in taxonomy.columns:
    if sum(taxonomy[c]) > 90:
        good_samples.append(c)
taxonomy_aux = taxonomy[good_samples]

From previous experiments we obtained better performance taking out the samples labeled as Obese, that is the reason we are not going to consider them either

In [77]:
obese = list(metadata[metadata['Diagnosis'] == 'Obese']['SampleID'])
healthy = list(metadata[metadata['Diagnosis'] == 'Healthy']['SampleID'])
non_healthy = list(metadata[metadata['Diagnosis'] != 'Healthy']['SampleID'])
non_healthy = [x for x in non_healthy if x not in obese]

Next we will define a function to perform Kolmogorov-Smirnov test to find the most important features for the PCA model

In [78]:
def ks_test(df, healthy, non_healthy, method_ks = 'auto', p_val = 0.001):
    healthy_df = df[[x for x in df.columns if x in healthy]].T
    nonhealthy_df = df[[x for x in df.columns if x in non_healthy]].T
    healthy_features = []
    nonhealthy_features = []
    for feature in list(df.index):
        if kstest(list(healthy_df[feature]), list(nonhealthy_df[feature]), alternative = 'less', method = method_ks).pvalue <= p_val:
            healthy_features.append(feature)
        if kstest(list(nonhealthy_df[feature]), list(healthy_df[feature]), alternative = 'less', method = method_ks).pvalue <= p_val:
            nonhealthy_features.append(feature)
    print(f'# Healthy features selected by KS: {len(healthy_features)}')
    print(f'# Unheatlhy features selected by KS: {len(nonhealthy_features)}')
    return healthy_features, nonhealthy_features

Now we define the data preprocessing workflow as defined in the paper

In [79]:
def custom_transform(x):
    if x <= 1:
        return np.log2(2 * x + 0.00001)
    else:
        return np.sqrt(x)

In [80]:
def transform_data(df, healthy_features, nonhealthy_features):
    scaler = StandardScaler()
    selected = df.T[[x for x in list(set(healthy_features + nonhealthy_features)) if x in df.T.columns]]
    selected = selected.applymap(custom_transform)

    for c in selected.columns:
        scaler.fit(np.array(selected[c]).reshape(-1, 1))
        selected[c] = scaler.transform(np.array(selected[c]).reshape(-1, 1))

    return selected

Then we define a function to perform PCA over the selected features only

In [81]:
def get_pca_data(df):
    pca = PCA()

    pca.fit(df)

    eigenvalues = pca.explained_variance_
    eigenvectors = pca.components_
    singular = pca.singular_values_
    
    pca_data = pd.DataFrame(zip(eigenvectors, eigenvalues, singular), columns = ('Eigenvectors', 'Explained_variance', 'Singular_values')).sort_values('Explained_variance', ascending = False)
    pca_data['%variance'] = pca_data['Explained_variance'] / sum(pca_data['Explained_variance'])
    pca_data = pca_data.sort_values('%variance', ascending = False)
    pca_data['%variance_cumulative'] = pca_data['%variance'].cumsum()
    
    return pca_data, pca

After that we start calculating the indexes, below there is a function to calculate T^2 index

In [82]:
def hotelling_t2(df, pca, pca_data, variance_for_pc = 0.9, alpha = 0.05):
    principal_components = list(pca_data[pca_data['%variance_cumulative'] < variance_for_pc]['Eigenvectors'])
    print(f'# Principal Components selected: {len(principal_components)}')
    principal_values = list(pca_data[pca_data['%variance_cumulative'] < variance_for_pc]['Explained_variance'])
    D = np.array(principal_components).T @ np.linalg.inv(np.diag(principal_values)) @ np.array(principal_components)
    deg_free = len(principal_components) 
    # alpha = 0.05
    t2_threshold = chi2.ppf(1-alpha, deg_free)
    T2 = []
    pred = []
    
    try:
        for item in pca.transform(df):
            index = item.T @ D @ item
            T2.append(index)
            if index > t2_threshold:
                pred.append('Unhealthy')
            else:
                pred.append('Healthy')
    except:
        for item in np.array(df):
            index = item.T @ D @ item
            T2.append(index)
            if index > t2_threshold:
                pred.append('Unhealthy')
            else:
                pred.append('Healthy')
            
    hoteling = pd.DataFrame(zip(df.index, T2, pred), columns = ['Sample', 'T2', 'Prediction T2'])
    
    return D, principal_components, hoteling, t2_threshold
    

Here we made a modification to Q_statistic limit according to ()

In [83]:
def Q_statistic(df, pca, pca_data, variance_for_pc = 0.9, alpha = 0.05):
    principal_components_residual = list(pca_data[pca_data['%variance_cumulative'] >= variance_for_pc]['Eigenvectors'])
    principal_values_residual = list(pca_data[pca_data['%variance_cumulative'] >= variance_for_pc]['Explained_variance'])
    principal_singvalues_residual = list(pca_data[pca_data['%variance_cumulative'] >= variance_for_pc]['Singular_values'])
    
    C = np.array(principal_components_residual).T @ np.array(principal_components_residual)
    Theta1 = sum(principal_values_residual)
    Theta2 = sum([x**2 for x in principal_values_residual])
    Theta3 = sum([x**3 for x in principal_values_residual])
    
    c_alpha = norm.ppf(1-alpha)
    
    h0 = 1-((2*Theta1*Theta3)/(3*Theta2**2))
    
    Q_alpha = Theta1*(((((c_alpha*np.sqrt(2*Theta2*(h0**2)))/Theta1)+1+((Theta2*h0*(h0-1))/(Theta1**2))))**(1/h0))
    
    Q = []
    pred = []
    try:
        for item in pca.transform(df):
            index = item.T @ C @ item
            Q.append(index)
            if index > Q_alpha:
                pred.append('Unhealthy')
            else:
                pred.append('Healthy')
    except:
        for item in np.array(df):
            index = item.T @ C @ item
            Q.append(index)
            if index > Q_alpha:
                pred.append('Unhealthy')
            else:
                pred.append('Healthy')
    
    Q_statistic = pd.DataFrame(zip(df.index, Q, pred), columns = ['Sample', 'Q', 'Prediction Q'])
    
    return C, Theta1, Theta2, Q_statistic, Q_alpha
    
    

In [84]:
def combined_index(df, D, t2_threshold, principal_components, Q_alpha, Theta1, Theta2, pca, alpha = 0.05):
    fi = D/t2_threshold + (np.eye(len(principal_components[0])) - (np.array(principal_components).T @ np.array(principal_components)))/Q_alpha
    g = ((len(principal_components) / t2_threshold**2) + (Theta2 / Q_alpha**2)) / ((len(principal_components)/t2_threshold) + (Theta1 / Q_alpha))
    h = ((len(principal_components)/t2_threshold) + (Theta1 / Q_alpha))**2 / ((len(principal_components) / t2_threshold**2) + (Theta2 / Q_alpha**2))

    chi_value = chi2.ppf(1-alpha, h)
    threshold_combined = g*chi_value
    combined = []
    pred = []
    
    try:
        for item in pca.transform(df):
            index = item.T @ fi @ item
            combined.append(index)
            if index > threshold_combined:
                pred.append('Unhealthy')
            else:
                pred.append('Healthy')
    except:
        for item in np.array(df):
            index = item.T @ fi @ item
            combined.append(index)
            if index > threshold_combined:
                pred.append('Unhealthy')
            else:
                pred.append('Healthy')

    combined = pd.DataFrame(zip(df.index, combined, pred), columns = ['Sample', 'Combined', 'Prediction Combined']) 
    return combined

Finally we define a function to calculate the index 

In [85]:
def hiPCA(df, healthy, non_health, df_ks = [], healthy_features = [], non_healthy_features = [], ks = False, method = 'auto', p_val = 0.001, only_nonhealthy_features = False):
    if ks:
        healthy_features, non_healthy_features = ks_test(df_ks, healthy, non_healthy, method_ks = method, p_val = p_val)
        
    if only_nonhealthy_features: 
        selected = transform_data(df, [], non_healthy_features)
    else:
        selected = transform_data(df, healthy_features, non_healthy_features)
        
    pca_data, pca = get_pca_data(selected)
    D, principal_components, results_hotelling, t2_threshold = hotelling_t2(selected, pca, pca_data)
    C, Theta1, Theta2, results_q, Q_alpha = Q_statistic(selected, pca, pca_data)
    hiPCA = combined_index(selected, D, t2_threshold, principal_components, Q_alpha, Theta1, Theta2, pca)
    
    return healthy_features, non_healthy_features, pd.concat([results_hotelling, results_q.drop('Sample', axis = 1),  hiPCA.drop('Sample', axis = 1)], axis=1, join='outer')

## Experiments

### Experiment 1

First we will calculate KS-hiPCA index to the trainning data and evaluate its result using balanced accuracy

In [86]:
healthy_features, non_healthy_features, results = hiPCA(taxonomy, healthy, non_healthy, df_ks = taxonomy_aux, ks = True, only_nonhealthy_features = True)

# Healthy features selected by KS: 140
# Unheatlhy features selected by KS: 22
# Principal Components selected: 15


In [87]:
original_hiPCA = balanced_accuracy_score(['Unhealthy' if x != 'Healthy' else 'Healthy' for x in metadata[metadata['SampleID'].isin(taxonomy.columns)]['Diagnosis']], ['Unhealthy' if x > 3.8 else 'Healthy' for x in metadata['hiPCA']])

In [88]:
ks_hiPCA = balanced_accuracy_score(['Unhealthy' if x != 'Healthy' else 'Healthy' for x in metadata[metadata['SampleID'].isin(taxonomy.columns)]['Diagnosis']],results['Prediction Combined'])

In [89]:
t2_hiPCA = balanced_accuracy_score(['Unhealthy' if x != 'Healthy' else 'Healthy' for x in metadata[metadata['SampleID'].isin(taxonomy.columns)]['Diagnosis']],results['Prediction T2'])

In [90]:
q_hiPCA = balanced_accuracy_score(['Unhealthy' if x != 'Healthy' else 'Healthy' for x in metadata[metadata['SampleID'].isin(taxonomy.columns)]['Diagnosis']],results['Prediction Q'])

In [91]:
print(f'Given hiPCA: {original_hiPCA}')
print(f'KS - hiPCA (T^2) : {t2_hiPCA}')
print(f'KS - hiPCA (Q) : {q_hiPCA}')
print(f'KS - hiPCA (Combined): {ks_hiPCA}')

Given hiPCA: 0.6726198083067092
KS - hiPCA (T^2) : 0.666038338658147
KS - hiPCA (Q) : 0.6999787007454739
KS - hiPCA (Combined): 0.7498509052183173


>Note that we did not have the threshold for the given hiPCA to classify samples, we arbitrarly chose the threshold that gave best result to make a fair comparison

Now we will use this model to try to predict the unhealthy samples

In [92]:
taxonomy_covid = pd.read_csv('../../DataSets/COVID/CAMDA_taxa.txt', sep = '\t', index_col = 0)

In [93]:
_, _, results_covid_taxonomy = hiPCA(taxonomy_covid, [], [], non_healthy_features = non_healthy_features)

# Principal Components selected: 10


In [94]:
results_covid_taxonomy.to_csv('../../output/hiPCA/kshiPCA_covid_taxonomy.csv', index = False)

### Experiment 2

In [95]:
unhealthy_tax = pd.read_csv('../../DataSets/INDEX/hiPCA/zhu_ks92_unhealthy_species.txt', sep="\t")

In [96]:
healthy_features, non_healthy_features, results = hiPCA(taxonomy, healthy, non_healthy, non_healthy_features = list(unhealthy_tax['species']))

# Principal Components selected: 35


In [97]:
hiPCA_zhutax = balanced_accuracy_score(['Unhealthy' if x != 'Healthy' else 'Healthy' for x in metadata[metadata['SampleID'].isin(taxonomy.columns)]['Diagnosis']],results['Prediction Combined'])
hiPCA_zhutax_t2 = balanced_accuracy_score(['Unhealthy' if x != 'Healthy' else 'Healthy' for x in metadata[metadata['SampleID'].isin(taxonomy.columns)]['Diagnosis']],results['Prediction T2'])
hiPCA_zhutax_q = balanced_accuracy_score(['Unhealthy' if x != 'Healthy' else 'Healthy' for x in metadata[metadata['SampleID'].isin(taxonomy.columns)]['Diagnosis']],results['Prediction Q'])

In [98]:
print(f'Given hiPCA: {original_hiPCA}')
print(f'hiPCA with Zhu taxa (T^2): {hiPCA_zhutax_t2}')
print(f'hiPCA with Zhu taxa (Q): {hiPCA_zhutax_q}')
print(f'hiPCA with Zhu taxa (Combined): {hiPCA_zhutax}')

Given hiPCA: 0.6726198083067092
hiPCA with Zhu taxa (T^2): 0.5877742279020234
hiPCA with Zhu taxa (Q): 0.6293503727369542
hiPCA with Zhu taxa (Combined): 0.647555910543131


In [99]:
_, _, results_covid_taxonomy = hiPCA(taxonomy_covid, [], [], non_healthy_features = list(unhealthy_tax['species']) )

# Principal Components selected: 15


In [100]:
results_covid_taxonomy.to_csv('../../output/hiPCA/hiPCAzhutax_covid.csv', index = False)

### Experiment 3

Now we tried to fit pathways data to the KS - hiPCA

In [101]:
pathways = pd.read_csv('../../DataSets/CAMDA/pathways.txt', sep = '\t', index_col = 0)

In [102]:
pathways = pathways.T[[x for x in pathways.index if 'UNINTEGRATED' not in x]]
pathways = pathways.T

In [103]:
good_samples = []
for c in taxonomy.columns:
    if sum(taxonomy[c]) > 90:
        good_samples.append(c)
pathways_aux = pathways[good_samples]

In [104]:
ob = list(metadata[metadata['Diagnosis'] == 'Obese']['SampleID'])

In [105]:
healthy = list(metadata[metadata['Diagnosis'] == 'Healthy']['SampleID'])
non_healthy = list(metadata[metadata['Diagnosis'] != 'Healthy']['SampleID'])
non_healthy = [x for x in non_healthy if x not in ob]

In [106]:
pathways_healthy = pathways[[x for x in pathways_aux.columns if x in healthy]].T
pathways_nonhealthy = pathways[[x for x in pathways_aux.columns if x in non_healthy]].T

In [107]:
healthy_features, non_healthy_features, results_pathways = hiPCA(pathways, healthy, non_healthy, df_ks = pathways_aux, ks = True, only_nonhealthy_features = False, method = 'asymp', p_val = 0.001)

# Healthy features selected by KS: 1103
# Unheatlhy features selected by KS: 770
# Principal Components selected: 28


In [108]:
ks_hiPCA_path = balanced_accuracy_score(['Unhealthy' if x != 'Healthy' else 'Healthy' for x in metadata[metadata['SampleID'].isin(taxonomy.columns)]['Diagnosis']],results_pathways['Prediction Combined'])
ks_hiPCA_patht2 = balanced_accuracy_score(['Unhealthy' if x != 'Healthy' else 'Healthy' for x in metadata[metadata['SampleID'].isin(taxonomy.columns)]['Diagnosis']],results_pathways['Prediction T2'])
ks_hiPCA_pathq = balanced_accuracy_score(['Unhealthy' if x != 'Healthy' else 'Healthy' for x in metadata[metadata['SampleID'].isin(taxonomy.columns)]['Diagnosis']],results_pathways['Prediction Q'])

In [109]:
print(f'Given hiPCA: {original_hiPCA}')
print(f'hiPCA with pathways (T^2): {ks_hiPCA_patht2}')
print(f'hiPCA with pathways (Q): {ks_hiPCA_pathq}')
print(f'hiPCA with pathways (Combined): {ks_hiPCA_path}')

Given hiPCA: 0.6726198083067092
hiPCA with pathways (T^2): 0.6078434504792332
hiPCA with pathways (Q): 0.512321618743344
hiPCA with pathways (Combined): 0.6063099041533546


In [110]:
pathways_covid = pd.read_csv('../../DataSets/COVID/CAMDA_pathways.txt', sep = '\t', index_col = 0)

In [111]:
_, _, results_hotelling_covid, results_q_covid, results_hiPCA_covid = hiPCA(pathways_covid, [], [], healthy_features = healthy_features, non_healthy_features = non_healthy_features)

ValueError: at least one array or dtype is required